# Машинное обучение с использованием Gold слоя и MLflow

В этом ноутбуке мы:
1. Загружаем данные из gold слоя
2. Подготавливаем данные для машинного обучения
3. Обучаем модель предсказания возраста
4. Оцениваем метрики модели
5. Логируем модель и параметры с использованием MLflow

## Цель: предсказание возраста пользователя по характеристикам устройства и поведению

In [ ]:
!pip install --upgrade pyspark==3.5.0 delta-spark

In [ ]:
!pip install mlflow==2.9.1

In [ ]:
!pip install --upgrade typing_extensions typing_inspection

In [12]:
# Импорт необходимых библиотек
import os
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta import *

# ML библиотеки Spark
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# MLflow для логирования
import mlflow
import mlflow.spark
from mlflow.tracking import MlflowClient

print("Импорт библиотек завершен успешно")

Импорт библиотек завершен успешно


In [13]:
# Инициализация Spark с оптимизациями
builder = (SparkSession.builder
    .appName("Age Predictor ML Model")
    .master("spark://spark-standalone:7077")
    .config("spark.driver.host", "jupyter-spark")
    .config("spark.driver.bindAddress", "0.0.0.0")
    .config("spark.driver.port", "7078")
    .config("spark.blockManager.port", "7079")
    
    # Delta Lake оптимизации
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    
    # Оптимизации производительности
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
)

# Создаем SparkSession
spark = configure_spark_with_delta_pip(builder).getOrCreate()

print("Spark session initialized")
print(f"Spark version: {spark.version}")
print(f"Available cores: {spark.sparkContext.defaultParallelism}")

Spark session initialized
Spark version: 3.5.0
Available cores: 2


In [14]:
# Настройка MLflow
# Устанавливаем tracking URI
mlflow.set_tracking_uri("http://mlflow-server:5000")

# Создаем или получаем эксперимент
experiment_name = "age-predictor-spark"
try:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Создан новый эксперимент: {experiment_name}")
except mlflow.exceptions.MlflowException:
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id
    print(f"Используется существующий эксперимент: {experiment_name}")

mlflow.set_experiment(experiment_name)
print(f"MLflow настроен, experiment_id: {experiment_id}")

Создан новый эксперимент: age-predictor-spark
MLflow настроен, experiment_id: 1


## Загрузка данных из Gold слоя

Загружаем очищенные данные из silver слоя, которые будут использованы для обучения модели.

In [15]:
# Загрузка основного датасета из silver слоя для обучения
silver_path = "/opt/data/silver"
gold_path = "/opt/data/gold"

# Загружаем основной датасет
df = spark.read.format("delta").load(f"{silver_path}/cleaned_dataset")

print(f"Загружено записей: {df.count()}")
print("\nСхема данных:")
df.printSchema()

print("\nПример данных:")
df.show(5)

Загружено записей: 19482

Схема данных:
root
 |-- user_id: long (nullable = true)
 |-- region_name: string (nullable = true)
 |-- city_name: string (nullable = true)
 |-- cpe_manufacturer_name: string (nullable = true)
 |-- cpe_model_name: string (nullable = true)
 |-- url_host: string (nullable = true)
 |-- cpe_type_cd: string (nullable = true)
 |-- cpe_model_os_type: string (nullable = true)
 |-- price: double (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- part_of_day: string (nullable = true)
 |-- request_cnt: long (nullable = true)
 |-- age: double (nullable = true)
 |-- is_male: string (nullable = true)


Пример данных:
+-------+-------------------+---------------+---------------------+-------------------+--------------------+-----------+-----------------+-------+-------------------+-----------+-----------+----+-------+
|user_id|        region_name|      city_name|cpe_manufacturer_name|     cpe_model_name|            url_host|cpe_type_cd|cpe_model_os_type|  price|  

In [16]:
# Загрузка дополнительных данных из gold слоя для обогащения признаков
print("Загрузка дополнительных данных из gold слоя...")

# Статистика по производителям
manufacturer_stats = spark.read.format("delta").load(f"{gold_path}/manufacturer_statistics")
print(f"Статистика по производителям: {manufacturer_stats.count()} записей")

# Статистика по регионам
region_stats = spark.read.format("delta").load(f"{gold_path}/region_statistics")
print(f"Статистика по регионам: {region_stats.count()} записей")

# Временные паттерны
time_patterns = spark.read.format("delta").load(f"{gold_path}/time_patterns")
print(f"Временные паттерны: {time_patterns.count()} записей")

print("\nДополнительные данные загружены")

Загрузка дополнительных данных из gold слоя...
Статистика по производителям: 29 записей
Статистика по регионам: 80 записей
Временные паттерны: 12 записей

Дополнительные данные загружены


## Подготовка данных для машинного обучения

Обогащаем основной датасет статистиками из gold слоя и подготавливаем признаки для обучения модели.

In [17]:
# Обогащение данных статистиками из gold слоя
print("Обогащение данных статистиками...")

# Добавляем статистику по производителям
manufacturer_features = manufacturer_stats.select(
    col("cpe_manufacturer_name"),
    col("avg_user_age").alias("manufacturer_avg_age"),
    col("avg_device_price").alias("manufacturer_avg_price"),
    col("male_percentage").alias("manufacturer_male_pct")
)

df_enriched = df.join(manufacturer_features, "cpe_manufacturer_name", "left")

# Добавляем статистику по регионам
region_features = region_stats.select(
    col("region_name"),
    col("avg_user_age").alias("region_avg_age"),
    col("avg_device_price").alias("region_avg_price"),
    col("ios_percentage").alias("region_ios_pct")
)

df_enriched = df_enriched.join(region_features, "region_name", "left")

# Добавляем временные паттерны
time_features = time_patterns.select(
    col("part_of_day"),
    col("cpe_model_os_type"),
    col("avg_user_age").alias("time_pattern_avg_age"),
    col("total_request_count").alias("time_pattern_requests")
)

df_enriched = df_enriched.join(
    time_features, 
    ["part_of_day", "cpe_model_os_type"], 
    "left"
)

print(f"Данные обогащены, записей: {df_enriched.count()}")
print(f"Колонок: {len(df_enriched.columns)}")

Обогащение данных статистиками...
Данные обогащены, записей: 19482
Колонок: 22


In [18]:
# Создание дополнительных признаков
print("Создание дополнительных признаков...")

df_features = df_enriched \
    .withColumn("price_log", log(col("price") + 1)) \
    .withColumn("request_cnt_log", log(col("request_cnt") + 1)) \
    .withColumn("price_per_request", col("price") / (col("request_cnt") + 1)) \
    .withColumn("is_expensive_device", when(col("price") > 30000, 1).otherwise(0)) \
    .withColumn("is_ios", when(col("cpe_model_os_type") == "iOS", 1).otherwise(0)) \
    .withColumn("is_male_int", col("is_male").cast(IntegerType()))

# Заполняем пропуски
df_features = df_features.fillna({
    "manufacturer_avg_age": 35.0,
    "manufacturer_avg_price": 20000.0,
    "manufacturer_male_pct": 50.0,
    "region_avg_age": 35.0,
    "region_avg_price": 20000.0,
    "region_ios_pct": 30.0,
    "time_pattern_avg_age": 35.0,
    "time_pattern_requests": 1000
})

print("Дополнительные признаки созданы")
print(f"Итоговых колонок: {len(df_features.columns)}")

Создание дополнительных признаков...
Дополнительные признаки созданы
Итоговых колонок: 28


In [19]:
# Фильтруем данные с валидным возрастом
df_ml = df_features.filter(
    (col("age").isNotNull()) & 
    (col("age") >= 14) & 
    (col("age") <= 80)
)

print(f"Данных для ML после фильтрации: {df_ml.count()}")

# Статистика по целевой переменной
age_stats = df_ml.select("age").describe()
print("\nСтатистика по возрасту:")
age_stats.show()

Данных для ML после фильтрации: 19474

Статистика по возрасту:
+-------+-----------------+
|summary|              age|
+-------+-----------------+
|  count|            19474|
|   mean|38.95060080106809|
| stddev| 11.7993470961305|
|    min|             14.0|
|    max|             80.0|
+-------+-----------------+



In [20]:
# Подготовка категориальных признаков
print("Подготовка категориальных признаков...")

categorical_columns = [
    "region_name",
    "cpe_manufacturer_name", 
    "cpe_model_os_type",
    "part_of_day"
]

# Создаем индексеры и энкодеры для категориальных переменных
indexers = []
encoders = []

for col_name in categorical_columns:
    indexer = StringIndexer(
        inputCol=col_name, 
        outputCol=f"{col_name}_index",
        handleInvalid="keep"
    )
    encoder = OneHotEncoder(
        inputCol=f"{col_name}_index", 
        outputCol=f"{col_name}_encoded"
    )
    indexers.append(indexer)
    encoders.append(encoder)

print(f"Подготовлено {len(categorical_columns)} категориальных признаков")

Подготовка категориальных признаков...
Подготовлено 4 категориальных признаков


In [21]:
# Определяем числовые признаки
numerical_features = [
    "price", "price_log", "request_cnt", "request_cnt_log", 
    "price_per_request", "is_expensive_device", "is_ios", "is_male_int",
    "manufacturer_avg_age", "manufacturer_avg_price", "manufacturer_male_pct",
    "region_avg_age", "region_avg_price", "region_ios_pct",
    "time_pattern_avg_age", "time_pattern_requests"
]

# Кодированные категориальные признаки
encoded_features = [f"{col}_encoded" for col in categorical_columns]

# Все признаки для модели
all_features = numerical_features + encoded_features

print(f"Числовых признаков: {len(numerical_features)}")
print(f"Категориальных признаков: {len(encoded_features)}")
print(f"Всего признаков: {len(all_features)}")

Числовых признаков: 16
Категориальных признаков: 4
Всего признаков: 20


## Создание ML Pipeline и обучение модели

Создаем пайплайн машинного обучения с обработкой признаков и обучением модели.

In [22]:
# Создание векторного ассемблера
vector_assembler = VectorAssembler(
    inputCols=all_features,
    outputCol="features",
    handleInvalid="skip"
)

# Создание модели Random Forest
rf = RandomForestRegressor(
    featuresCol="features",
    labelCol="age",
    numTrees=100,
    maxDepth=10,
    seed=42
)

# Создание полного пайплайна
pipeline_stages = indexers + encoders + [vector_assembler, rf]
pipeline = Pipeline(stages=pipeline_stages)

print(f"Пайплайн создан с {len(pipeline_stages)} этапами")

Пайплайн создан с 10 этапами


In [23]:
# Разделение данных на train/test
print("Разделение данных на train/test...")

# Кэшируем данные для ускорения
df_ml.cache()

train_data, test_data = df_ml.randomSplit([0.8, 0.2], seed=42)

print(f"Обучающая выборка: {train_data.count()} записей")
print(f"Тестовая выборка: {test_data.count()} записей")

# Кэшируем разделенные данные
train_data.cache()
test_data.cache()

Разделение данных на train/test...
Обучающая выборка: 15693 записей
Тестовая выборка: 3781 записей


DataFrame[part_of_day: string, cpe_model_os_type: string, region_name: string, cpe_manufacturer_name: string, user_id: bigint, city_name: string, cpe_model_name: string, url_host: string, cpe_type_cd: string, price: double, date: timestamp, request_cnt: bigint, age: double, is_male: string, manufacturer_avg_age: double, manufacturer_avg_price: double, manufacturer_male_pct: double, region_avg_age: double, region_avg_price: double, region_ios_pct: double, time_pattern_avg_age: double, time_pattern_requests: bigint, price_log: double, request_cnt_log: double, price_per_request: double, is_expensive_device: int, is_ios: int, is_male_int: int]

In [24]:
# Настройка гиперпараметров для кросс-валидации
print("Настройка гиперпараметров...")

param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [50, 100]) \
    .addGrid(rf.maxDepth, [8, 10]) \
    .addGrid(rf.subsamplingRate, [0.8, 1.0]) \
    .build()

# Создание оценщика
evaluator = RegressionEvaluator(
    labelCol="age",
    predictionCol="prediction",
    metricName="rmse"
)

# Кросс-валидация
cv = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=param_grid,
    evaluator=evaluator,
    numFolds=3,
    seed=42
)

print(f"Настроена кросс-валидация с {len(param_grid)} комбинациями параметров")

Настройка гиперпараметров...
Настроена кросс-валидация с 8 комбинациями параметров


In [26]:
# Обучение модели с логированием в MLflow
with mlflow.start_run(run_name="RandomForest_Age_Prediction") as run:
    
    print("Начало обучения модели...")
    
    # Логируем параметры датасета
    mlflow.log_param("train_size", train_data.count())
    mlflow.log_param("test_size", test_data.count())
    mlflow.log_param("num_features", len(all_features))
    mlflow.log_param("numerical_features", len(numerical_features))
    mlflow.log_param("categorical_features", len(categorical_columns))
    
    # Обучение модели
    cv_model = cv.fit(train_data)
    
    # Получаем лучшую модель
    best_model = cv_model.bestModel
    best_rf_model = best_model.stages[-1]  # Random Forest - последний этап пайплайна

    num_trees        = best_rf_model.getNumTrees
    max_depth        = best_rf_model.getMaxDepth
    subsampling_rate = best_rf_model.getSubsamplingRate

    # Логируем лучшие параметры
    mlflow.log_param("best_numTrees",        num_trees)
    mlflow.log_param("best_maxDepth",        max_depth)
    mlflow.log_param("best_subsamplingRate", subsampling_rate)

    print("Модель обучена успешно!")
    print(f"  numTrees: {num_trees}")
    print(f"  maxDepth: {max_depth}")
    print(f"  subsamplingRate: {subsampling_rate}")

    run_id = run.info.run_id
    print("Run ID:", run_id)

Начало обучения модели...
Модель обучена успешно!
  numTrees: 100
  maxDepth: <bound method _DecisionTreeParams.getMaxDepth of RandomForestRegressionModel: uid=RandomForestRegressor_1c10ed6dad8d, numTrees=100, numFeatures=131>
  subsamplingRate: <bound method _TreeEnsembleParams.getSubsamplingRate of RandomForestRegressionModel: uid=RandomForestRegressor_1c10ed6dad8d, numTrees=100, numFeatures=131>
Run ID: 9bc38cc56a7f4ac08b1e3e2ba3ab9ac7


## Оценка модели и логирование метрик

In [27]:
# Предсказания на тестовых данных
print("Выполнение предсказаний...")

test_predictions = best_model.transform(test_data)
train_predictions = best_model.transform(train_data)

# Вычисление метрик
rmse_evaluator = RegressionEvaluator(labelCol="age", predictionCol="prediction", metricName="rmse")
mae_evaluator = RegressionEvaluator(labelCol="age", predictionCol="prediction", metricName="mae")
r2_evaluator = RegressionEvaluator(labelCol="age", predictionCol="prediction", metricName="r2")

# Метрики на тестовых данных
test_rmse = rmse_evaluator.evaluate(test_predictions)
test_mae = mae_evaluator.evaluate(test_predictions)
test_r2 = r2_evaluator.evaluate(test_predictions)

# Метрики на обучающих данных
train_rmse = rmse_evaluator.evaluate(train_predictions)
train_mae = mae_evaluator.evaluate(train_predictions)
train_r2 = r2_evaluator.evaluate(train_predictions)

# Логируем метрики
mlflow.log_metric("test_rmse", test_rmse)
mlflow.log_metric("test_mae", test_mae)
mlflow.log_metric("test_r2", test_r2)
mlflow.log_metric("train_rmse", train_rmse)
mlflow.log_metric("train_mae", train_mae)
mlflow.log_metric("train_r2", train_r2)

# Логируем разницу (переобучение)
mlflow.log_metric("overfitting_rmse", train_rmse - test_rmse)
mlflow.log_metric("overfitting_mae", train_mae - test_mae)

print("\n=== МЕТРИКИ МОДЕЛИ ===")
print(f"Тестовые данные:")
print(f"  RMSE: {test_rmse:.4f}")
print(f"  MAE:  {test_mae:.4f}")
print(f"  R²:   {test_r2:.4f}")

print(f"\nОбучающие данные:")
print(f"  RMSE: {train_rmse:.4f}")
print(f"  MAE:  {train_mae:.4f}")
print(f"  R²:   {train_r2:.4f}")

print(f"\nПереобучение:")
print(f"  RMSE разница: {train_rmse - test_rmse:.4f}")
print(f"  MAE разница:  {train_mae - test_mae:.4f}")

Выполнение предсказаний...

=== МЕТРИКИ МОДЕЛИ ===
Тестовые данные:
  RMSE: 11.0599
  MAE:  8.9692
  R²:   0.1077

Обучающие данные:
  RMSE: 10.6720
  MAE:  8.6414
  R²:   0.1757

Переобучение:
  RMSE разница: -0.3880
  MAE разница:  -0.3278


In [28]:
# Анализ важности признаков
print("\nАнализ важности признаков...")

feature_importances = best_rf_model.featureImportances.toArray()

# Создаем список признаков с их важностью
feature_importance_data = []
for i, importance in enumerate(feature_importances):
    if i < len(all_features):
        feature_importance_data.append((all_features[i], float(importance)))

# Сортируем по важности
feature_importance_data.sort(key=lambda x: x[1], reverse=True)

print("\nТоп-10 самых важных признаков:")
for i, (feature, importance) in enumerate(feature_importance_data[:10]):
    print(f"{i+1:2d}. {feature:<30} {importance:.4f}")
    mlflow.log_metric(f"feature_importance_{feature}", importance)

# Логируем важность топ-5 признаков отдельно
for i, (feature, importance) in enumerate(feature_importance_data[:5]):
    mlflow.log_metric(f"top_{i+1}_feature_importance", importance)


Анализ важности признаков...

Топ-10 самых важных признаков:
 1. price                          0.1681
 2. price_log                      0.1133
 3. is_expensive_device            0.0934
 4. is_male_int                    0.0832
 5. region_avg_age                 0.0769
 6. price_per_request              0.0587
 7. region_ios_pct                 0.0396
 8. manufacturer_male_pct          0.0362
 9. time_pattern_requests          0.0245
10. time_pattern_avg_age           0.0211


In [ ]:
# Сохранение модели в MLflow
print("\nСохранение модели в MLflow...")

# TODO: Добавить права на папку. Сейчас ошибка сохранения из-за отсутвия прав на папку /mlartifacts

# Логируем модель
mlflow.spark.log_model(
    spark_model=best_model,
    artifact_path="artifacts",
    registered_model_name="age-predictor-random-forest"
)

# Сохраняем дополнительную информацию
model_info = {
    "model_type": "RandomForestRegressor",
    "framework": "PySpark MLlib",
    "target_variable": "age",
    "features_used": all_features,
    "data_source": "gold_layer"
}

mlflow.log_dict(model_info, "model_info.json")

print(f"Модель сохранена в MLflow с run_id: {run.info.run_id}")

# Сохраняем run_id для дальнейшего использования
run_id = run.info.run_id

In [48]:
# Временный обходной путь :)
tmp_model_path = "/home/jovyan/data/age_rf_model"
best_model.write().overwrite().save(tmp_model_path)
print("Модель сохранена локально в:", tmp_model_path)

Модель сохранена локально в: /home/jovyan/data/age_rf_model


## Дополнительная оценка модели по сегментам

In [30]:
# Анализ производительности модели по разным сегментам
print("Анализ производительности по сегментам...")

# По операционным системам
print("\nПроизводительность по операционным системам:")
os_performance = test_predictions.groupBy("cpe_model_os_type") \
    .agg(
        count("*").alias("count"),
        avg(abs(col("age") - col("prediction"))).alias("avg_abs_error"),
        avg(col("age")).alias("avg_actual_age"),
        avg(col("prediction")).alias("avg_predicted_age")
    )

os_performance.show()

# По ценовым сегментам
print("\nПроизводительность по ценовым сегментам:")
price_segments_performance = test_predictions \
    .withColumn("price_segment", 
        when(col("price") < 10000, "budget")
        .when(col("price") < 30000, "mid_range")
        .when(col("price") < 50000, "premium")
        .otherwise("luxury")
    ) \
    .groupBy("price_segment") \
    .agg(
        count("*").alias("count"),
        avg(abs(col("age") - col("prediction"))).alias("avg_abs_error"),
        avg(col("age")).alias("avg_actual_age"),
        avg(col("prediction")).alias("avg_predicted_age")
    )

price_segments_performance.show()

# По полу
print("\nПроизводительность по полу:")
gender_performance = test_predictions.groupBy("is_male") \
    .agg(
        count("*").alias("count"),
        avg(abs(col("age") - col("prediction"))).alias("avg_abs_error"),
        avg(col("age")).alias("avg_actual_age"),
        avg(col("prediction")).alias("avg_predicted_age")
    )

gender_performance.show()

Анализ производительности по сегментам...

Производительность по операционным системам:
+-----------------+-----+-----------------+-----------------+-----------------+
|cpe_model_os_type|count|    avg_abs_error|   avg_actual_age|avg_predicted_age|
+-----------------+-----+-----------------+-----------------+-----------------+
|              iOS| 1114|8.512885823181312|35.10592459605027| 34.8760403826548|
|        Apple iOS|   12|12.84349364646937|            41.25|33.74091721330873|
|          Android| 2473|9.155901845042736|40.45208249090174|40.86519560978756|
+-----------------+-----+-----------------+-----------------+-----------------+


Производительность по ценовым сегментам:
+-------------+-----+-----------------+------------------+-----------------+
|price_segment|count|    avg_abs_error|    avg_actual_age|avg_predicted_age|
+-------------+-----+-----------------+------------------+-----------------+
|       luxury|  662|7.780339108962179| 34.14803625377643|34.17288443249355|
|

## Финализация и очистка

In [31]:
# Анализ предсказаний
print("\nАнализ предсказаний...")

# Примеры предсказаний
sample_predictions = test_predictions.select(
    "age", "prediction", "cpe_manufacturer_name", 
    "price", "region_name", "is_male"
).limit(10)

print("\nПримеры предсказаний на тестовых данных:")
sample_predictions.show(10, truncate=False)

# Статистика по ошибкам
predictions_with_error = test_predictions.withColumn(
    "error", abs(col("age") - col("prediction"))
)

error_stats = predictions_with_error.select("error").describe()
print("\nСтатистика по абсолютным ошибкам:")
error_stats.show()

# Логируем дополнительные метрики
error_percentiles = predictions_with_error.approxQuantile("error", [0.5, 0.75, 0.9, 0.95], 0.01)
mlflow.log_metric("error_median", error_percentiles[0])
mlflow.log_metric("error_75th_percentile", error_percentiles[1])
mlflow.log_metric("error_90th_percentile", error_percentiles[2])
mlflow.log_metric("error_95th_percentile", error_percentiles[3])


Анализ предсказаний...

Примеры предсказаний на тестовых данных:
+----+------------------+---------------------+-------+----------------+-------+
|age |prediction        |cpe_manufacturer_name|price  |region_name     |is_male|
+----+------------------+---------------------+-------+----------------+-------+
|39.0|41.036427301988404|Huawei               |6990.0 |Алтайский край  |1      |
|67.0|41.0383782526698  |Huawei               |7992.0 |Алтайский край  |1      |
|39.0|36.56842325505082 |Huawei               |20657.0|Алтайский край  |1      |
|42.0|40.600735591301145|Huawei               |13755.0|Алтайский край  |0      |
|23.0|40.19540808469163 |Samsung              |17439.0|Алтайский край  |1      |
|34.0|37.40326114093041 |Samsung              |30421.0|Алтайский край  |1      |
|21.0|40.89294136162046 |Samsung              |3990.0 |Алтайский край  |1      |
|58.0|44.478360045895904|Xiaomi               |9451.0 |Алтайский край  |0      |
|62.0|44.61973242517967 |Huawei            

In [40]:
# Сводка по эксперименту
print("\n" + "="*50)
print("СВОДКА ПО ML")
print("="*50)
print(f"MLflow Run ID: {run_id}")
print(f"Модель: RandomForestRegressor")
print(f"Данные: Gold слой Age Predictor")
print(f"Размер обучающей выборки: {train_data.count():,}")
print(f"Размер тестовой выборки: {test_data.count():,}")
print(f"Количество признаков: {len(all_features)}")
print(f"Лучшие параметры:")
print(f"  - NumTrees: {best_rf_model.getNumTrees}")
print(f"  - MaxDepth: {best_rf_model.getMaxDepth()}")
print(f"  - SubsamplingRate: {best_rf_model.getSubsamplingRate()}")
print(f"Итоговые метрики:")
print(f"  - Test RMSE: {test_rmse:.4f}")
print(f"  - Test MAE: {test_mae:.4f}")
print(f"  - Test R²: {test_r2:.4f}")
print("="*50)

# Логируем сводные метрики
mlflow.log_metric("final_score", test_r2)
mlflow.log_metric("model_complexity", best_rf_model.getNumTrees * best_rf_model.getMaxDepth())

# Завершаем MLflow run
print("\nMLflow run завершен успешно!")


СВОДКА ПО ML
MLflow Run ID: 9bc38cc56a7f4ac08b1e3e2ba3ab9ac7
Модель: RandomForestRegressor
Данные: Gold слой Age Predictor
Размер обучающей выборки: 15,693
Размер тестовой выборки: 3,781
Количество признаков: 20
Лучшие параметры:
  - NumTrees: 100
  - MaxDepth: 8
  - SubsamplingRate: 0.8
Итоговые метрики:
  - Test RMSE: 11.0599
  - Test MAE: 8.9692
  - Test R²: 0.1077

MLflow run завершен успешно!


In [49]:
# Освобождение памяти и очистка
print("Освобождение памяти...")

# Отменяем кэширование
df_ml.unpersist()
train_data.unpersist()
test_data.unpersist()

# Очистка кэша Spark
spark.catalog.clearCache()

print("Память освобождена, кэш очищен")

Освобождение памяти...
Память освобождена, кэш очищен


In [50]:
# Остановка Spark сессии
print("Остановка Spark сессии...")
spark.stop()
print("Spark сессия остановлена")

print("\n" + "="*60)
print("ML ЗАВЕРШЕН УСПЕШНО!")
print("="*60)
print("Модель обучена и сохранена в MLflow")
print("Метрики и параметры залогированы")
print("Важность признаков проанализирована")
print("Производительность по сегментам оценена")
print("Ресурсы освобождены")
print("="*60)

Остановка Spark сессии...
Spark сессия остановлена

ML ЗАВЕРШЕН УСПЕШНО!
Модель обучена и сохранена в MLflow
Метрики и параметры залогированы
Важность признаков проанализирована
Производительность по сегментам оценена
Ресурсы освобождены
